In [2]:
import os
import argparse

import torch
#from tokenization_hanbert import HanBertTokenizer
import pandas as pd
import numpy as np
import collections
import h5py
#from dataloader import KoreanDataset, KoreanBatchGenerator, get_dataloader
from tqdm.notebook import tqdm


In [3]:
from bart.tokenization_hanbert import HanBertTokenizer
tokenizer = HanBertTokenizer.from_pretrained("bart/HanBart-54kN")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'HanBertTokenizer'.


In [2]:
test = """시대가 변함에 따라 아동 폭력의 양상도 바뀌고 있다. 어른에 의해 발생하는 폭력뿐 아니라 또래관계에서 발생하는 폭력도 증가하고 있다. 특히, 인터넷 매체를 통한 언어적 괴롭힘 등 사이버 공간이 새로운 피해현장으로 등장했다. 피해현장에 놓여있는 아이들이 이런 사회변화의 심각성을 적극적으로 알리고 있다. \u200b아동옹호대표기관 초록우산어린이재단(회장 이제훈)은 국제어린이재단연맹(ChildFund Alliance) 국가와 함께 아동폭력을 근절하기 위한 지속적인 노력을 이어나가고 있다. 대한민국에서는 초록우산어린이재단 서울아동옹호센터에서 2018년부터 아동폭력 옹호사업(Child Friendly Accountability, 이하 아동폭력 옹호사업)을 진행하며 아동이 스스로 아동폭력의 심각성을 알려 변화를 촉구하는 옹호활동을 이어나가고 있다. \u200b이 활동의 일환으로, 지난 7월 10일(뉴욕 현지시간) 국제어린이재단연맹 중 아동폭력 옹호사업을 진행하는 4개의 국가(엘살바도르, 우간다, 파라과이, 대한민국)의 대표아동들이 UN을 방문했다. 해당 아동들은 UN HLPF(High Level Political Forum, 이하 고위급정치회담)에 참여하여 아동폭력의 심각성과 국제사회의 관심 및 해결을 촉구하는 시간을 가졌다. \u200b대한민국 대표로 참석한 초록우산어린이재단 이윤서 아동(신수중 3)은 '우리나라에서는 아동폭력이 다른 나라와는 조금 다른 모습으로 나타나고 있다'며 세 가지 아동폭력 상황에 대해 설명했다. \u200b대한민국에서 발생하는 아동폭력 상황은 먼저, 어른들이 교육, 훈육의 모습으로 아동에게 가하는 체벌에 대한 내용이다. 최근 정부에서 발표한 '포용국가 아동정책' 중에는 가정 내 체벌을 없애기 위해 민법상 친권자의 징계권을 개정하겠다는 내용이 포함됐다. 포럼에 참여한 이윤서 아동은 정부의 정책과 연결하여 '아이를 부모의 소유물이라고 생각하는 우리나라 사회 분위기가 먼저 바뀌어야 한다'며 인식개선의 필요성에 대한 의견을 전했다. \u200b두 번째로 또래관계에서 발생하는 신체, 언어, 사이버폭력 심각하게는 성폭력을 예로 들었다. '학교 내에서는 또래관계에서 발생하는 폭력을 해결하기 위한 안전장치가 마련되어 있지만 실효성이 없다'며 대책 마련의 필요성을 공유했다. \u200b마지막으로 지나친 학업과 좋은 성적을 중시하는 사회적 분위기를 꼽았다. '과도한 학업 경쟁으로 인해 한 학원에서는 부모의 동의를 구하고 체벌을 감행하는 경우도 있다'며 '치열해지는 경쟁 속에 친구를 경쟁자로 인식하게 만드는 사회 분위기가 변해야 한다'는 의견을 전했다. \u200b이외에도 본 포럼에 함께 참여한 각 국의 아동대표들은 적극적인 목소리로 아동노동, 조혼, 인신매매 등 아동폭력 심각성을 알렸다. \u200bUN 고위급정치회담(HLPF)은 2016년 9월, UN이 수립한 SDGs(Sustainable Development Goals, 이하 지속가능개발목표) 의제를 점검하는 회의로, 매년 7월 뉴욕에 위치한 UN 본부에서 진행된다. 특히 2019년은 '아동폭력 근절을 위한 전 지구적 협력 방안 모색(Building Momentum: working together for all children to live free from violence)'이라는 주제로 각국 정부 관계자, UN관계자, 학계, 시민사회 인사 등이 참석하여 아동들의 목소리가 더욱 의미 있게 전달될 수 있었다. \u200bUN에서 진행된 고위급정치회담(HLPF) 일정을 함께 한 초록우산어린이재단 국제협력개발1본부 이성호본부장은 '올해는 유엔아동권리협약 비준 30주년을 맞이하는 뜻 깊은 해'라며 '우리의 현재이자, 지속 가능한 발전의 초석이 되는 아동이 직접 참여하여 이번 포럼이 더욱 뜻 깊었다'는 소감을 전했다. \u200b초록우산어린이재단은 국제어린이재단연맹(ChildFund Alliance) 회원국과 함께 SDGs 16.2(지속가능개발목표, 아동폭력) 이슈를 해결하기 위해 UN 회원국가, UN 기관 등을 대상으로 지속적인 옹호활동을 전개해 나갈 예정이다."""

In [4]:
test

"시대가 변함에 따라 아동 폭력의 양상도 바뀌고 있다. 어른에 의해 발생하는 폭력뿐 아니라 또래관계에서 발생하는 폭력도 증가하고 있다. 특히, 인터넷 매체를 통한 언어적 괴롭힘 등 사이버 공간이 새로운 피해현장으로 등장했다. 피해현장에 놓여있는 아이들이 이런 사회변화의 심각성을 적극적으로 알리고 있다. \u200b아동옹호대표기관 초록우산어린이재단(회장 이제훈)은 국제어린이재단연맹(ChildFund Alliance) 국가와 함께 아동폭력을 근절하기 위한 지속적인 노력을 이어나가고 있다. 대한민국에서는 초록우산어린이재단 서울아동옹호센터에서 2018년부터 아동폭력 옹호사업(Child Friendly Accountability, 이하 아동폭력 옹호사업)을 진행하며 아동이 스스로 아동폭력의 심각성을 알려 변화를 촉구하는 옹호활동을 이어나가고 있다. \u200b이 활동의 일환으로, 지난 7월 10일(뉴욕 현지시간) 국제어린이재단연맹 중 아동폭력 옹호사업을 진행하는 4개의 국가(엘살바도르, 우간다, 파라과이, 대한민국)의 대표아동들이 UN을 방문했다. 해당 아동들은 UN HLPF(High Level Political Forum, 이하 고위급정치회담)에 참여하여 아동폭력의 심각성과 국제사회의 관심 및 해결을 촉구하는 시간을 가졌다. \u200b대한민국 대표로 참석한 초록우산어린이재단 이윤서 아동(신수중 3)은 '우리나라에서는 아동폭력이 다른 나라와는 조금 다른 모습으로 나타나고 있다'며 세 가지 아동폭력 상황에 대해 설명했다. \u200b대한민국에서 발생하는 아동폭력 상황은 먼저, 어른들이 교육, 훈육의 모습으로 아동에게 가하는 체벌에 대한 내용이다. 최근 정부에서 발표한 '포용국가 아동정책' 중에는 가정 내 체벌을 없애기 위해 민법상 친권자의 징계권을 개정하겠다는 내용이 포함됐다. 포럼에 참여한 이윤서 아동은 정부의 정책과 연결하여 '아이를 부모의 소유물이라고 생각하는 우리나라 사회 분위기가 먼저 바뀌어야 한다'며 인식개선의 필요성에 대한 의견을 전했다. \u200b두 번

In [69]:
with open('sharding_corpus/bartcorpus_000.txt', 'r') as f:
    data = f.readlines()

In [70]:
len(data)

281431

In [24]:
more = 0
for c_index, context in enumerate(data):
    a = tokenizer.encode_plus(context,
                #padding='max_length', #'max_length'
                #max_length=512,
                #truncation=True,
                #add_special_tokens=False,
                return_tensors='np')['input_ids'][0]
    if len(a)>512:
        more+=1
        print(more,c_index)

1 3
2 23
3 34
4 35
5 41
6 51
7 54
8 58
9 60
10 70
11 74
12 75
13 76
14 80
15 88
16 89
17 95
18 112
19 139
20 144
21 154
22 155
23 160
24 164
25 165
26 168
27 170
28 172
29 173
30 179
31 180
32 193
33 196
34 201
35 202
36 205
37 208
38 211
39 218
40 220
41 227
42 229
43 230
44 231
45 235
46 237
47 239
48 240
49 241
50 242
51 244
52 247
53 250
54 252
55 258
56 264
57 267
58 269
59 271
60 276
61 286
62 291
63 293
64 297
65 298
66 304
67 309
68 310
69 320
70 325
71 326
72 328
73 330
74 342
75 365
76 367
77 374
78 375
79 378
80 379
81 386
82 391
83 392
84 402
85 410
86 417
87 422
88 429
89 436
90 440
91 448
92 449
93 453
94 454
95 455
96 457
97 462
98 463
99 466
100 467
101 475
102 478
103 480
104 484
105 486
106 487
107 488
108 492
109 494
110 495
111 498
112 499
113 507
114 509
115 512
116 514
117 519
118 521
119 538
120 539
121 575
122 579
123 584
124 596
125 600
126 604
127 626
128 637
129 647
130 654
131 658
132 663
133 669
134 677
135 682
136 684
137 685
138 692
139 696
140 698
141 71

KeyboardInterrupt: 

In [21]:
data[0:4]

["중기청과 중기중앙회는 지난 1일 청와대 영빈관에서 각계 중소기업인 200여 명이 함께 하는 '2014 대한민국 중소기업인대회'를 개최했다. 어려운 여건에도 불구하고 경제 활성화에 기여하고 있는 중소기업인을 격려하기 위해 마련된 자리다. 이날 행사에서 박근혜 대통령은 각 분야에서 성공을 일궈온 모범 중소기업인과 중소기업 지원 역할을 모범적으로 수행한 단체를 격려했다. 특히 박 대통령은 포상(은탑산업훈장)이 확정됐으나 지난 5월 말 별세한 고(故) 여우균 전 화남피혁 대표의 부인에게 훈장을 대리수여해 산업발전에 기여한 고인의 공로를 기렸다.\n",
 "중기청과 대ㆍ중소기업협력재단은 기술자료 임치 1만건 돌파를 기념해 지난 3일 서울 여의도 중소기업중앙회에서 '2014 중소기업기술 보호 세미나'를 개최했다. 이날 행사에선 기술임치제도 활성화에 기여한 중소기업 3곳과 대기업 2곳에 각각 중소기업청장상과 동반성장위원장 공로패를 수여했다. 이 자리에선 삼성전자, 현대차 등 국내 5개 대기업과 중기청, 동반성장위원회 간에 '중소기업기술 보호 가이드라인 준수'를 위한 협약도 체결됐다.\n",
 '중소기업진흥공단에서는 중국 시안, 베트남 하노이 글로벌비즈니스인큐베이터(BI) 입주 업체를 모집한다. 시안은 중국 중ㆍ서부 내수시장 진출을 위한 요충지로서 대ㆍ중소기업 동반 진출이 이뤄지고 있으며 하노이도 현재 삼성전자 협력사 90여 개, LG전자 협력사 10여 개가 위치하는 등 투자가 활발한 지역이다. 홈페이지(www.sbc-kbdc.com)를 통해 연중 신청 가능하다.\n',
 '건설사들이 트렌드를 반영한 새로운 평면을 속속 선보이면서 신평면 아파트에 대한 인기가 뜨겁다. 특히 올해 평면경쟁의 화두는 실속으로 각 건설사들마다 부담은 줄이고 주거가치를 끌어올리는 것이 특징이다. 올해 건설사들이 새로 내놓고 있는 평면들은 초소형부터 3~4인가구에 적합한 중형 평면까지 다양하다. 복층형 펜트하우스 등과 같이 이전처럼 완전히 다른 평면들을 내놓는 것은 아니지만 전체 면적은 줄이는 대

In [35]:
a = tokenizer.batch_encode_plus(data[0:4],
        padding='max_length', #'max_length'
        max_length=512,
        truncation=True,
        #add_special_tokens=False,
        return_tensors='pt')

In [58]:
attention_mask=a['attention_mask']

In [59]:
attention_mask.sum(-1).detach().tolist()

[145, 107, 102, 512]

In [61]:
attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [46]:
input_ids=a['input_ids']

In [6]:
def shift_tokens_right(input_ids, pad_token_id, eos_token_id):
    """
        Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
        decoder input: <eos><sos> tok1 tok2 … tokn
        target:<sos> tok1 tok2 … tokn <eos>
        https://github.com/huggingface/transformers/issues/7961
    """
    prev_output_tokens = input_ids.clone()
    #index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = torch.tensor(eos_token_id)
    prev_output_tokens[:, 1:] = input_ids[:, :-1]

    last_index = prev_output_tokens.ne(pad_token_id).sum(dim=1)-1
    for i, last_i in enumerate(last_index):
        prev_output_tokens[i][last_i] = eos_token_id
    return prev_output_tokens        

In [50]:
prev_output_tokens = input_ids.clone()
#index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
prev_output_tokens[:, 0] = torch.tensor(tokenizer.eos_token_id)
prev_output_tokens[:, 1:] = input_ids[:, :-1]

In [54]:
tokenizer.decode(prev_output_tokens[3])

'[EOS] [CLS] 건설사들이 트렌드를 반영한 새로운 평면을 속속 선보이면서 신평면 아파트에 대한 인기가 뜨겁다. 특히 올해 평면경쟁의 화두는 실속으로 각 건설사들마다 부담은 줄이고 주거가치를 끌어올리는 것이 특징이다. 올해 건설사들이 새로 내놓고 있는 평면들은 초소형부터 3 4인가구에 적합한 중형 평면까지 다양하다. 복층형 펜트하우스 등과 같이 이전처럼 완전히 다른 평면들을 내놓는 것은 아니지만 전체 면적은 줄이는 대신 수납공간과 서비스 면적 등을 늘려 실제 생활에 꼭 필요한 요소들을 첨가했다. 평면개발도 ` 실속형 + α\'로 진화하고 있는 것이다. 건설사들이 실속평면 개발에 힘을 쏟는 이유는 지속되고 있는 경기불황으로 주택시장이 실수요자 위주로 재편됐기 때문이다. 또한 1 2인 또는 3인 가구의 증가로 기존의 획일화된 면적에 대한 선호도가 떨어진 것도 이유다. 한 건설사 관계자는 " 소형 4,5베이 등 지난해 인기를 끌었던 평면들을 볼 때 이미 건설사들 사이에서는 기존의 평범한 평면으로는 현 주택시장에서 살아남기 힘들다는 공감대가 형성돼 있다 " 며 " 특히 실수요자들의 경우 실생활에 적합한 평면인지를 꼼꼼히 따지기 때문에 평면개발에 더욱 노력을 기울일 수 밖에 없다 " 고 말했다. 실제로 건설사들이 발표하고 있는 신평면들은 기존의 단점을 극복하고 공간 활용을 극대화하는 것에 초점을 맞추고 있다. 특히 반도건설이 경기 평택시에서 선보인 ` 소사벌 반도유보라 아이비파크\'도 실속형 평면특화로 수요자들의 눈길을 사로 잡고 있다. 특히 아파트 전용면적 [UNK] 타입은 활용성을 극대화한 수납공간 및 공간 활용의 혁신 설계를 적용해 인기가 높다. 가변형 벽체 적용으로 입주민의 라이프 스타일에 맞춰 거실과 침실로 분리 또는 합쳐 대형 거실 ( 약 7.3m 광폭 거실 ) 을 구성할 수 있다. 또한 주부를 위한 공간도 설계된다. 대형 펜트리와 수납장을 극대화 시켜 주방 공간 등을 더 넓고 효율적으로 사용할 수 있게 한다. 또한 활용성을 극대화한 대형 드레스룸이 조

In [55]:
tokenizer.decode(input_ids[3])

'[CLS] 건설사들이 트렌드를 반영한 새로운 평면을 속속 선보이면서 신평면 아파트에 대한 인기가 뜨겁다. 특히 올해 평면경쟁의 화두는 실속으로 각 건설사들마다 부담은 줄이고 주거가치를 끌어올리는 것이 특징이다. 올해 건설사들이 새로 내놓고 있는 평면들은 초소형부터 3 4인가구에 적합한 중형 평면까지 다양하다. 복층형 펜트하우스 등과 같이 이전처럼 완전히 다른 평면들을 내놓는 것은 아니지만 전체 면적은 줄이는 대신 수납공간과 서비스 면적 등을 늘려 실제 생활에 꼭 필요한 요소들을 첨가했다. 평면개발도 ` 실속형 + α\'로 진화하고 있는 것이다. 건설사들이 실속평면 개발에 힘을 쏟는 이유는 지속되고 있는 경기불황으로 주택시장이 실수요자 위주로 재편됐기 때문이다. 또한 1 2인 또는 3인 가구의 증가로 기존의 획일화된 면적에 대한 선호도가 떨어진 것도 이유다. 한 건설사 관계자는 " 소형 4,5베이 등 지난해 인기를 끌었던 평면들을 볼 때 이미 건설사들 사이에서는 기존의 평범한 평면으로는 현 주택시장에서 살아남기 힘들다는 공감대가 형성돼 있다 " 며 " 특히 실수요자들의 경우 실생활에 적합한 평면인지를 꼼꼼히 따지기 때문에 평면개발에 더욱 노력을 기울일 수 밖에 없다 " 고 말했다. 실제로 건설사들이 발표하고 있는 신평면들은 기존의 단점을 극복하고 공간 활용을 극대화하는 것에 초점을 맞추고 있다. 특히 반도건설이 경기 평택시에서 선보인 ` 소사벌 반도유보라 아이비파크\'도 실속형 평면특화로 수요자들의 눈길을 사로 잡고 있다. 특히 아파트 전용면적 [UNK] 타입은 활용성을 극대화한 수납공간 및 공간 활용의 혁신 설계를 적용해 인기가 높다. 가변형 벽체 적용으로 입주민의 라이프 스타일에 맞춰 거실과 침실로 분리 또는 합쳐 대형 거실 ( 약 7.3m 광폭 거실 ) 을 구성할 수 있다. 또한 주부를 위한 공간도 설계된다. 대형 펜트리와 수납장을 극대화 시켜 주방 공간 등을 더 넓고 효율적으로 사용할 수 있게 한다. 또한 활용성을 극대화한 대형 드레스룸이 조성된다. 여

In [41]:
prev_output_tokens = shift_tokens_right(a['input_ids'], tokenizer.pad_token_id, tokenizer.eos_token_id)

In [42]:
prev_output_tokens

tensor([[    1,     3, 31046,  ...,     0,     0,     0],
        [    1,     3, 31046,  ...,     0,     0,     0],
        [    1,     3, 15795,  ...,     0,     0,     0],
        [    1,     3, 19106,  ...,   935, 38155,     1]])

In [24]:
tokenizer.pad_token_id

PreTrainedTokenizer(name_or_path='bart/HanBart-54kN', vocab_size=54000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

False

In [6]:
import subprocess
import os

In [7]:
shard_file_index=0

In [11]:
sharding_path = '/home/jisu/sharding_corpus/'
h5py_path = "/home/jisu/h5py_dataset2/"


In [14]:
files = os.listdir(sharding_path)

In [17]:
with open(sharding_path+files[0], 'r') as f:
    text = f.readlines()

'/home/jisu/sharding_corpus/bartcorpus_161.txt'

In [20]:
num_divided=281600


In [23]:
n_processes = 8
def create_record_worker(shard_file_index):
    num_divided=203269

    sharding_filename = f"bartcorpus_{str(shard_file_index).zfill(3)}.txt"
    h5py_filename = f"h5py_data_{str(shard_file_index).zfill(3)}"

    bart_preprocessing_command = 'python make_h5py.py'
    bart_preprocessing_command += ' --input_file=' + sharding_path+sharding_filename
    bart_preprocessing_command += ' --output_file=' + h5py_path+h5py_filename+'.h5py'
    bart_preprocessing_command += ' --num_divided=' + str(num_divided)


    bert_preprocessing_process = subprocess.Popen(bart_preprocessing_command, shell=True)

    last_process = bert_preprocessing_process

    # This could be better optimized (fine if all take equal time)
    if shard_file_index % n_processes == 0 and shard_file_index > 0:
        bert_preprocessing_process.wait()
    return last_process


In [24]:
last_process =create_record_worker(255)


In [22]:

for i in range(len(files)):
    print(i)
    last_process =create_record_worker(i)

last_process.wait()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


0